In [181]:
import os, shutil
import random
import tensorflow as tf
from tensorflow.keras import (models, layers, optimizers)

import plotly.express as ex
import plotly.graph_objects as go

ImportError: Plotly express requires pandas to be installed.

In [161]:
data_path = "../data/raw"
x_train = os.listdir(data_path + "/train")
# x_test = os.listdir(data_path + "/test1")
x_train = [os.path.join(data_path, "train", x) for x in x_train]
# x_test = [os.path.join(data_path, "test1", x) for x in x_test]
print(len(x_train))
# print(len(x_test))
print(x_train[:5])
# print(x_test[:5])

25000
['../data/raw/train/dog.11082.jpg', '../data/raw/train/cat.2955.jpg', '../data/raw/train/cat.12060.jpg', '../data/raw/train/dog.971.jpg', '../data/raw/train/cat.6966.jpg']


In [162]:
x_train = random.choices(x_train, k=3000)
# x_test = random.choices(x_train, k=250)
print(len(x_train))
# print(len(x_test))

3000


In [163]:
data_path = "../data/processed"
if os.path.exists(data_path):
    shutil.rmtree(data_path)
os.makedirs(data_path + "/train/dogs")
os.makedirs(data_path + "/train/cats")

In [164]:
for x in x_train:
    if "dog" in x:
        shutil.copy(x, data_path + "/train/dogs")
    else:
        shutil.copy(x, data_path + "/train/cats")

In [165]:
print(len(os.listdir(data_path + "/train/dogs")))
print(len(os.listdir(data_path + "/train/cats")))

1393
1433


In [166]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_56 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_56 (MaxPoolin  (None, 74, 74, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_57 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_57 (MaxPoolin  (None, 36, 36, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_58 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_58 (MaxPoolin  (None, 17, 17, 128)    

In [167]:
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    metrics=['acc']
)

In [168]:
batch_size = 16
x_train, x_valid = tf.keras.utils.image_dataset_from_directory(
    (data_path + "/train"),
    batch_size=batch_size,
    image_size=(150, 150),
    validation_split=.2,
    subset="both",
    seed=42
)

Found 2826 files belonging to 2 classes.
Using 2261 files for training.
Using 565 files for validation.


In [179]:
x_train_count = len(x_train) * batch_size
x_valid_count = len(x_valid) * batch_size
print(x_train_count)
print(x_valid_count)

2272
576


In [180]:
history = model.fit(
    x_train.repeat(),
    steps_per_epoch=int(x_train_count/batch_size),
    epochs=25,
    validation_data=x_valid.repeat(),
    validation_steps=int(x_valid_count/batch_size)
)

Epoch 1/25
142/142 [==============================] - 15s 102ms/step - loss: 3.1457 - acc: 0.5334 - val_loss: 0.8552 - val_acc: 0.5310
Epoch 2/25
142/142 [==============================] - 15s 107ms/step - loss: 0.6750 - acc: 0.6210 - val_loss: 0.7022 - val_acc: 0.5823
Epoch 3/25
142/142 [==============================] - 16s 111ms/step - loss: 0.5688 - acc: 0.7001 - val_loss: 0.8899 - val_acc: 0.5699
Epoch 4/25
142/142 [==============================] - 16s 114ms/step - loss: 0.4712 - acc: 0.7616 - val_loss: 0.8624 - val_acc: 0.6230
Epoch 5/25
142/142 [==============================] - 17s 118ms/step - loss: 0.3765 - acc: 0.8266 - val_loss: 1.5638 - val_acc: 0.5310
Epoch 6/25
142/142 [==============================] - 16s 115ms/step - loss: 0.3172 - acc: 0.8571 - val_loss: 0.8915 - val_acc: 0.6655
Epoch 7/25
117/142 [=======================>......] - ETA: 2s - loss: 0.2440 - acc: 0.8964

KeyboardInterrupt: 